# Operando con datos en Pandas

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas elementos a elemento, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.).

Pandas hereda gran parte de esta funcionalidad de NumPy. Sin embargo, incluye un par de cosas extra:
  - Para **operaciones unarias**, como funciones de negación y trigonométricas, estas funciones **mantendrán las etiquetas de índice y columna** en la salida.
  - Para **operaciones binarias**, como la suma y la multiplicación, Pandas automáticamente **alineará índices** pasando los objetos a la función.
  
Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes, que serían tareas propensas a errores con matrices NumPy sin procesar, se vuelven infalibles con Pandas. Además, veremos que hay operaciones bien definidas entre estructuras unidimensionales ``Series`` y estructuras bidimensionales ``DataFrame``.

## Funciones universales: Manteniendo índices

Una función universal (o ufunc para abreviar) es una función que puede ser usada en arrays de Numpy de forma nativa, siendo compatible con la difusión de matrices, la conversión de tipos y otras características estándar. Es decir, un ufunc es un contenedor "vectorizado" para una función que toma un número fijo de entradas escalares y produce un número fijo de salidas escalares.

Dado que Pandas está diseñado para trabajar con NumPy, cualquier ufunc de NumPy funcionará con ``Series`` y ``DataFrames`` de Pandas.

Veeamos un ejemplo con ambos objetos:

In [7]:
import pandas as pd
import numpy as np

In [8]:
np.random.seed(42)
ser = pd.Series(np.random.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [9]:
df = pd.DataFrame(np.random.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Si aplicamos una ufunc NumPy en cualquiera de estos objetos, el resultado será otro objeto Pandas **manteniendo los índices**:

In [10]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

O, para una operación un poco más compleja:

In [48]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## Funciones universales: alineación de índices

Para operaciones binarias con objetos Pandas, se alinearán los índices al realizar la operación, lo cual es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos siguientes:

### Alineación de índices en Series

Por ejemplo, supongamos que combinamos 2 Series de datos, una con los áreas de los 3 estados más extensos de USA, y otra con los 3 estados más poblados:

In [33]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')
area

Alaska        1723337
Texas          695662
California     423967
Name: area, dtype: int64

In [34]:
population

California    38332521
Texas         26448193
New York      19651127
Name: population, dtype: int64

Veamos qué ocurre cuando calculamos la densidad de población:

In [35]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

La matriz resultante contiene la unión de los índices de las dos matrices de entrada, que podrían determinarse utilizando la aritmética de conjuntos estándar de Python que hemos visto antes:

In [36]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Cualquier elemento para el que no se tenga una entrada (en cualqueira de los dos Series) se marca como ``NaN`` (Not a Number), que es la forma en que Pandas marca los datos faltantes.

Esta coincidencia de índices se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; los valores faltantes se rellenan con NaN de forma predeterminada:

In [37]:
s1 = pd.Series([2, 4, 6], index=[0, 1, 2])
s2 = pd.Series([1, 3, 5], index=[1, 2, 3])
s1 + s2

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

Si rellenar con NaN por defecto no es lo que queremos, podemos rellenar estos valores con lo que queramos usando los métodos adecuados en lugar de los operadores.

Por ejemplo, ejecutar a ``A.add(B)`` es equivalente a ejecutar ``A + B``, pero permite un parámetro opcional donde especificar de manera explícita con qué rellenar los valores para los elementos que falten de ``A`` o ``B``:

In [38]:
s1.add(s2, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Alineamiento de índices en DataFrames

Cuando se realizan operaciones con ``DataFrames``, se produce un tipo similar de alineación para columnas e índices:

In [39]:
df1 = pd.DataFrame(np.random.randint(0, 20, (2, 2)),
                 columns=list('AB'))
df1

,A,B
0,11,7
1,14,2


In [40]:
df2 = pd.DataFrame(np.random.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
df2

,B,A,C
0,0,3,1
1,7,3,1
2,5,5,9


In [41]:
df1 + df2

,A,B,C
0,14.0,7.0,NaN
1,17.0,9.0,NaN
2,NaN,NaN,NaN


Nótese que los índices están alineados correctamente independientemente de su orden en los dos objetos, siendo ordenados en el resultado final. Tal como hemos visto con las `` Series``,  podemos usar el método aritmético del objeto asociado y pasar cualquier ``fill_value`` deseado para usar en lugar de las entradas faltantes. Aquí rellenaremos con la media de todos los valores en ``A`` (calculados apilando primero las filas de ``A``):

In [46]:
fill = 8.5
df3 = df1.add(df2, fill_value=fill)
df1

,A,B
0,11,7
1,14,2


La siguiente tabla resume los operadores de Python y sus métodos de objeto Pandas equivalentes, que nos permitirán el uso de ciertos parámetros con los que ampliar la funcionalildad de los mismos:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Funciones universales: Operaciones entre Series y DataFrames

Al realizar operaciones entre un ``DataFrame`` y una ``Serie``, la alineación de índice y columnas se mantiene de manera similar. Las operaciones entre un ``DataFrame`` y una ``Serie`` son similares a las operaciones entre una matriz NumPy bidimensional y otra unidimensional.

Consideremos una operación común, donde calculamos la diferencia entre una matriz bidimensional y una de sus filas:

In [47]:
np.random.seed(42)
A = np.random.randint(10, size=(3, 4))
A

array([[6, 3, 7, 4],
       [6, 9, 2, 6],
       [7, 4, 3, 7]])

In [51]:
A - A[0]

array([[ 0,  0,  0,  0],
       [ 0,  6, -5,  2],
       [ 1,  1, -4,  3]])

De acuerdo con las reglas de difusión de NumPy, la resta entre una matriz bidimensional y una de sus filas se realiza por filas.

En Pandas, la convención opera de manera similar por filas de forma predeterminada:

In [58]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,0,6,-5,2
2,1,1,-4,3


Si, en cambio, queremos operar a nivel columna, podemos usar los métodos mencionados anteriormente, donde podremos especificar cómo operar mediante el parámetro ``axis``:

In [63]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,3,0,4,1
1,-3,0,-7,-3
2,3,0,-1,3


Nótese que las operaciones entre estos 2 objetos de Pandas ``DataFrame``/``Series``, como las vistas arriba, alinearán automáticamente los índices:

In [64]:
halfrow = df.iloc[0, ::2]
halfrow

Q    6
S    7
Name: 0, dtype: int32

In [ ]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


¿Qué acaba de ocurrir? ¿Por qué tenemos NaN en las columnas R y S?

La respuesta se basa en la alineación que veíamos anteriormente. Podemos entender la forma de operar de Pandas como que primero se crea los 2 dataframes que va a restar, que en este caso será el DataFrame ```df``` menos un nuevo DataFrame cuyas columnas Q y S serán las mismas que ``df`` pero que el resto de columnas no están definidas, por lo que se rellenarán con NaN. Al hacer la diferencia, cualquier resta que implique un NaN devolverá un NaN (por definición), por lo que las columnas R y T, pese a tener valores en ``df``, terminarán siendo todo NaN en el resultado final, obligado por el segundo DataFrame (que son NaN).

Si seleccionásemos la primera fila pero todas las columnas, no habría que rellenar nada con NaN, por lo que se restaría perfectamente esa fila a todas y cada una de las filas de ``df``, como hemos visto en apartados anteriores.

In [78]:
halfrow = df.iloc[0, ::]
df - halfrow

,Q,R,S,T
0,0,0,0,0
1,0,6,-5,2
2,1,1,-4,3


Esta preservación y alineación de índices y columnas, significa que las operaciones con datos en Pandas siempre mantendrán el contexto de datos, lo que evita los de errores sencillos que pueden surgir al trabajar con datos heterogéneos y/o desalineados en matrices NumPy sin procesar.

Por lo tanto, si dominamos esto, podremos manejar a nuestro antojo toda esta serie de operaciones. Así que... ¡¡vamos a hacer ejercicios!!